<a href="https://colab.research.google.com/github/rootdrew27/cyberbullying-ml/blob/main/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CatBoost ImplementationC

In [ ]:
!pip install catboost

In [ ]:
!pip install tensorflow

In [11]:
import pandas as pd
import numpy as np

import re

from sklearn.model_selection import train_test_split

import tensorflow as tf

from catboost import CatBoostClassifier, Pool, metrics, cv

In [74]:
df = pd.read_csv("./clean_data_2.csv", usecols=[1,2])

In [13]:
df.head()

,tweet_text,cyberbullying_type
0,in other words your food was crapilicious,0
1,why is so white,0
2,<@> a classy whore or more red velvet cupcakes,0
3,<@> gio meh p thanks for the heads up but not ...,0
4,<@> this is an isis account pretending to be a...,0


In [75]:
df.dropna(axis=0, inplace=True) #drop rows that contain any null values

In [76]:
df.reset_index(drop=True)

,tweet_text,cyberbullying_type
0,in other words your food was crapilicious,0
1,why is so white,0
2,<@> a classy whore or more red velvet cupcakes,0
3,<@> gio meh p thanks for the heads up but not ...,0
4,<@> this is an isis account pretending to be a...,0
...,...,...
43843,black people are not expected to do anything d...,4
43844,turner did not withhold his disappointment tur...,4
43845,i swear to god this dumb nigger bitch i have g...,4
43846,yea fuck you rt <@> if you are a nigger fuckin...,4


In [78]:
def remove_mentions(text):
  return re.sub(r'<@>', r'', text)

# add a mentions count column, and a tweet length column
df['mentions_count'] = [tweet.split().count('<@>') for tweet in df.tweet_text] #number of mentions in tweet
df['tweet_text'] = df['tweet_text'].apply(remove_mentions)
df['tweet_len'] = [len(tweet.split()) for tweet in df.tweet_text] #count words in tweet
df = df[df['tweet_len'] >= 2]
df.head()

,tweet_text,cyberbullying_type,mentions_count,tweet_len
0,in other words your food was crapilicious,0,0,7
1,why is so white,0,0,4
2,a classy whore or more red velvet cupcakes,0,1,8
3,gio meh p thanks for the heads up but not too...,0,1,18
4,this is an isis account pretending to be a ku...,0,1,17


# Split the Data

In [91]:
X = df.drop('cyberbullying_type', axis=1)
Y = df['cyberbullying_type']
#Y = tf.keras.utils.to_categorical(df['cyberbullying_type'], 6, 'int')

In [92]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=42)

array([[0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

# Preprocessing (Vectorization)

In [21]:
#get max tweet length (in words)
max_length = max([len(tweet.split()) for tweet in df.tweet_text])

In [38]:
# encoder = tf.keras.layers.TextVectorization(
#     output_mode="int",
#     output_sequence_length=max_length
# )
# encoder.adapt(x_train['tweet_text'])
# x_train.tweet_text = encoder.call(x_train.tweet_text).numpy()

In [93]:
y_train.shape

(30583,)

In [94]:
#Try pooling

train_pool = Pool(data=x_train,
                  label=y_train,
                  text_features=['tweet_text']
                  )
test_pool = Pool(data=x_test,
                 label=y_test,
                 text_features=['tweet_text'])

In [98]:
def fit_model(train_pool, test_pool, **kwargs):
    model = CatBoostClassifier(
        iterations=100,
        learning_rate=0.1,
        eval_metric='MultiClass',
        **kwargs
    )

    return model.fit(train_pool)

model = fit_model(train_pool, test_pool)

0:	learn: 1.4988112	total: 1.8s	remaining: 2m 57s
1:	learn: 1.3221193	total: 3.63s	remaining: 2m 58s
2:	learn: 1.1978454	total: 5.45s	remaining: 2m 56s
3:	learn: 1.1021841	total: 7.2s	remaining: 2m 52s
4:	learn: 1.0284873	total: 9.41s	remaining: 2m 58s
5:	learn: 0.9640448	total: 11.9s	remaining: 3m 7s
6:	learn: 0.9066418	total: 13.6s	remaining: 3m 1s
7:	learn: 0.8608973	total: 15.6s	remaining: 2m 59s
8:	learn: 0.8212844	total: 17.4s	remaining: 2m 55s
9:	learn: 0.7854392	total: 19.6s	remaining: 2m 56s
10:	learn: 0.7530646	total: 21.6s	remaining: 2m 54s
11:	learn: 0.7270892	total: 24.3s	remaining: 2m 57s
12:	learn: 0.7004746	total: 26.2s	remaining: 2m 55s
13:	learn: 0.6759905	total: 28.1s	remaining: 2m 52s
14:	learn: 0.6578233	total: 30.1s	remaining: 2m 50s
15:	learn: 0.6421278	total: 31.9s	remaining: 2m 47s
16:	learn: 0.6251047	total: 33.8s	remaining: 2m 45s
17:	learn: 0.6108959	total: 36.3s	remaining: 2m 45s
18:	learn: 0.5991620	total: 38.5s	remaining: 2m 44s
19:	learn: 0.5865280	total

In [100]:
y_pred = model.predict(test_pool)

In [101]:
from catboost.utils import eval_metric

acc = eval_metric(y_test, y_pred, 'Accuracy')[0]
print(acc)

0.8703844980164784
